In [1]:
import pandas as pd
import spacy

ModuleNotFoundError: No module named 'spacy'

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def get_similar_words(word):
    word = nlp(word)
    similar_words = []
    for token in nlp.vocab:
        if token.is_lower and token.has_vector and token.vector_norm:
            similarity = word.similarity(token)
            if similarity > 0.6:
                similar_words.append((token.text, similarity))
    return similar_words

print(get_similar_words("happy"))

In [3]:
words=["Whisper,
"Prism",
"Tumble",
"Glisten",
"Radiant",
"Bluster",
"Cascade",
"Drift",
"Clamor",
"Murmur",
"Glare",
"Dapple",
"Gleam",
"Flicker",
"Sparkle",
"Hush"

SyntaxError: unterminated string literal (detected at line 1) (3252203369.py, line 1)

In [13]:
df=pd.read_csv("explicitsim353.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'explicitsim353.csv'

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#words = ["radical","power"]
words=["dog","toilet"]
vectorizer = TfidfVectorizer().fit_transform(words)
vectors = vectorizer.toarray()
cosine_sim = cosine_similarity(vectors)
print(cosine_sim)

[[1. 0.]
 [0. 1.]]


In [17]:
import pandas as pd

In [19]:
df=pd.read_csv("explicitsim353.csv")

In [21]:
df

,cs_word_1,cs_word_2,average
0,Arafat,Jackson,2.051282
1,Arafat,peace,2.575000
2,Arafat,terror,3.125000
3,FBI,fingerprint,3.210526
4,FBI,investigation,4.789474
...,...,...,...
348,water,seepage,3.450000
349,weapon,secret,2.378378
350,weather,forecast,5.275000
351,wood,forest,6.710526


In [25]:
df.sort_values(by="average", ascending=False)

,cs_word_1,cs_word_2,average
334,tiger,tiger,9.375000
46,car,automobile,8.600000
196,money,cash,8.512821
195,money,cash,8.128205
189,midday,noon,8.025641
...,...,...,...
149,holy,sex,1.225000
92,delay,racism,1.175000
316,sugar,approach,1.128205
110,drink,ear,1.100000
